In [11]:
# =========================
# Assignment Q1 (CSV version) — Colab-ready
# =========================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- (A) Upload your CSV ----------
try:
    from google.colab import files  # type: ignore
    print("Upload your PWT 9.1 CSV (e.g., 'pwt91 - Copy.csv') ...")
    uploaded = files.upload()
    PWT_PATH = next(iter(uploaded.keys()))
except Exception:
    # Fallback if not in Colab; set path manually
    PWT_PATH = 'pwt91.csv'

print("Using file:", PWT_PATH)

# ---------- (B) Utils ----------
FIG_DIR = 'figures'
os.makedirs(FIG_DIR, exist_ok=True)

def read_csv_robust(path):
    """
    Read CSV with robust encoding & delimiter handling.
    Tries multiple encodings and lets pandas sniff the delimiter.
    """
    encodings = ['utf-8', 'utf-8-sig', 'latin1', 'ISO-8859-1', 'windows-1252']
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, sep=None, engine='python', encoding=enc)
            print(f"Loaded with encoding='{enc}' and sniffed delimiter.")
            return df
        except Exception as e:
            last_err = e
    raise last_err

def add_const(x):
    """Add a constant column (ones) to 1D/2D numpy array."""
    x = np.asarray(x)
    if x.ndim == 1:
        x = x.reshape(-1, 1)
    return np.column_stack([np.ones(x.shape[0]), x])

def ols_via_matrices(X, y):
    """
    OLS via β = (X'X)^{-1} X'y (computed with lstsq for stability).
    Returns (beta, yhat, resid, R^2).
    """
    X = np.asarray(X); y = np.asarray(y).reshape(-1, 1)
    beta, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    yhat = X @ beta
    resid = y - yhat
    sst = float(np.sum((y - y.mean())**2))
    sse = float(np.sum(resid**2))
    r2 = np.nan if sst == 0 else (1 - sse/sst)
    return beta.flatten(), yhat.flatten(), resid.flatten(), float(r2)

def keep_positive_for_logs(df, cols):
    """Return rows where all specified columns are strictly > 0."""
    mask = np.ones(len(df), dtype=bool)
    for c in cols:
        mask &= (pd.to_numeric(df[c], errors='coerce') > 0)
    return df.loc[mask].copy()

# ---------- (C) Load + prepare 2011 slice ----------
df_all = read_csv_robust(PWT_PATH)
# Clean header whitespace
df_all.columns = [c.strip() for c in df_all.columns]

# Required columns per assignment
cols_needed = ['country', 'rgdpe', 'avh', 'cn', 'pop', 'ctfp', 'year']

# Try to fix minor header issues (spaces/case)
if not set(cols_needed).issubset(df_all.columns):
    rename_map = {}
    for need in cols_needed:
        # loose matching (ignores spaces and case)
        candidates = [c for c in df_all.columns
                      if c.lower().replace(' ', '') == need.lower()]
        if candidates:
            rename_map[candidates[0]] = need
    if rename_map:
        df_all = df_all.rename(columns=rename_map)

missing = [c for c in cols_needed if c not in df_all.columns]
if missing:
    raise ValueError(f"Missing required columns after load: {missing}\n"
                     f"Available columns (first 25): {list(df_all.columns)[:25]}")

# Ensure numeric types where needed
for col in ['rgdpe','avh','cn','pop','ctfp','year']:
    df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

# Filter to year 2011 and keep only assignment columns
df = df_all.loc[df_all['year'] == 2011, cols_needed].copy()

# Drop rows with missing values in required variables
df = df.dropna(subset=['country','rgdpe','avh','cn','pop','ctfp']).reset_index(drop=True)

# Derived variables: GDP per capita and Capital per capita
df['gdp_pc'] = df['rgdpe'].astype(float) / df['pop'].astype(float)
df['cap_pc'] = df['cn'].astype(float) / df['pop'].astype(float)

print(f"Observations (2011, complete): {len(df)}")
try:
    from google.colab import data_table  # type: ignore
    data_table.DataTable(df.head(10), include_index=False)
except Exception:
    display(df.head())

# ---------- (D) Scatter plots (levels) ----------
plots_info = [
    ('ctfp',   'TFP (ctfp)',                     'scatter_gdppc_ctfp.png'),
    ('cap_pc', 'Capital per capita (cn/pop)',    'scatter_gdppc_cappc.png'),
    ('avh',    'Average annual hours (avh)',     'scatter_gdppc_avh.png'),
]

for xcol, xlabel, fname in plots_info:
    plt.figure()
    plt.scatter(df[xcol], df['gdp_pc'], s=14)
    plt.xlabel(xlabel); plt.ylabel('GDP per capita')
    plt.title(f'GDP per capita vs {xlabel} (2011)')
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, fname), dpi=150)
    plt.close()

# ---------- (E) Multivariate OLS in levels ----------
# Model: gdp_pc ~ const + ctfp + cap_pc + avh
Y = df['gdp_pc'].to_numpy()
X = add_const(df[['ctfp', 'cap_pc', 'avh']].to_numpy())
beta_levels, yhat_levels, resid_levels, r2_levels = ols_via_matrices(X, Y)

print("\n=== Multivariate OLS (levels) ===")
print("Model: gdp_pc = b0 + b1*ctfp + b2*cap_pc + b3*avh")
print("beta (b0, b1, b2, b3):", beta_levels)
print(f"R^2: {r2_levels:.4f}")

# ---------- (F) Three separate log–log OLS ----------
# Keep rows strictly positive for logs
df_logs = keep_positive_for_logs(df, ['gdp_pc','ctfp','cap_pc','avh'])
df_logs['lngdp_pc'] = np.log(df_logs['gdp_pc'])
df_logs['lnctfp']   = np.log(df_logs['ctfp'])
df_logs['lncap_pc'] = np.log(df_logs['cap_pc'])
df_logs['lnavh']    = np.log(df_logs['avh'])

print(f"\nObservations for log–log (all positive): {len(df_logs)}")

def simple_loglog(df_, xcol, ycol='lngdp_pc'):
    x = df_[xcol].to_numpy()
    y = df_[ycol].to_numpy()
    X1 = add_const(x)
    b, yhat, _, r2 = ols_via_matrices(X1, y)
    return b, yhat, r2

results = {}
for xcol, pretty in [('lnctfp','TFP (ctfp)'),
                     ('lncap_pc','Capital per capita (cn/pop)'),
                     ('lnavh','Hours worked (avh)')]:
    b, yhat, r2 = simple_loglog(df_logs, xcol)
    results[xcol] = (b, r2)

    # Scatter + fitted line in logs
    plt.figure()
    plt.scatter(df_logs[xcol], df_logs['lngdp_pc'], s=14, label='Data')
    xs = np.linspace(df_logs[xcol].min(), df_logs[xcol].max(), 200)
    ys = b[0] + b[1]*xs
    plt.plot(xs, ys, label='Fitted (log–log)')
    plt.xlabel(xcol); plt.ylabel('lngdp_pc')
    plt.title(f'ln(GDP per capita) vs {pretty}\n$R^2$={r2:.3f}')
    plt.legend()
    plt.tight_layout()
    outname = f'loglog_{xcol}.png'
    plt.savefig(os.path.join(FIG_DIR, outname), dpi=150)
    plt.close()

print("\n=== Simple log–log OLS ===")
for k,(b,r2) in results.items():
    print(f"{k}: beta0={b[0]:.4f}, beta1={b[1]:.4f}, R^2={r2:.4f}")

# ---------- (G) Save cleaned 2011 slice ----------
df[['country','rgdpe','avh','cn','pop','ctfp','gdp_pc','cap_pc']].to_csv('pwt2011_clean.csv', index=False)
print("\nSaved outputs:")
print(" - figures/scatter_gdppc_ctfp.png")
print(" - figures/scatter_gdppc_cappc.png")
print(" - figures/scatter_gdppc_avh.png")
print(" - figures/loglog_lnctfp.png")
print(" - figures/loglog_lncap_pc.png")
print(" - figures/loglog_lnavh.png")
print(" - pwt2011_clean.csv")


Upload your PWT 9.1 CSV (e.g., 'pwt91 - Copy.csv') ...


Saving pwt91 - Copy.csv to pwt91 - Copy (1).csv
Using file: pwt91 - Copy (1).csv
Loaded with encoding='latin1' and sniffed delimiter.
Observations (2011, complete): 61

=== Multivariate OLS (levels) ===
Model: gdp_pc = b0 + b1*ctfp + b2*cap_pc + b3*avh
beta (b0, b1, b2, b3): [-1.83341691e+04  1.85275808e+04  1.93712924e-01  5.56281285e+00]
R^2: 0.8929

Observations for log–log (all positive): 61

=== Simple log–log OLS ===
lnctfp: beta0=10.7399, beta1=1.8135, R^2=0.5066
lncap_pc: beta0=1.0623, beta1=0.7883, R^2=0.8981
lnavh: beta0=28.9782, beta1=-2.5080, R^2=0.2695

Saved outputs:
 - figures/scatter_gdppc_ctfp.png
 - figures/scatter_gdppc_cappc.png
 - figures/scatter_gdppc_avh.png
 - figures/loglog_lnctfp.png
 - figures/loglog_lncap_pc.png
 - figures/loglog_lnavh.png
 - pwt2011_clean.csv
